In [1]:
#(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()
import numpy as np
import pandas as pd
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
# Commonly used modules
import numpy as np
import os
import sys
# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2
import IPython
from six.moves import urllib
from sklearn.metrics import f1_score

print(tf.__version__)
from PIL import Image
from google.colab import drive
drive.mount('/content/drive')
if not os.path.exists("./output/") : os.mkdir("./output/")

2.7.0
Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/MLHW4DATA/HW4_data.zip

In [3]:
X_train = []
for i in range(10000):
    X_train.append(np.asarray(Image.open("HW4_data/public/train_{i}.jpg".format(i=i))))

df_train = pd.read_csv("HW4_data/public/train_label.csv", index_col=0)

X_val = []
for i in range(1000):
    X_val.append(np.asarray(Image.open("HW4_data/public/val_{i}.jpg".format(i=i))))

df_val = pd.read_csv("HW4_data/public/val_label.csv", index_col=0)

X_test = []
for i in range(1500):
    X_test.append(np.asarray(Image.open("HW4_data/private/test_{i}.jpg".format(i=i))))

df_test_demo = pd.read_csv("HW4_data/private/test_demo.csv", index_col=0)

X_train = np.asarray(X_train, dtype=np.float32)
X_val = np.asarray(X_val,dtype=np.float32)  
X_test = np.asarray(X_test,dtype=np.float32)

X_train = X_train.reshape(X_train.shape[0], 128*128)
X_val = X_val.reshape(X_val.shape[0], 128*128)
X_test = X_test.reshape(X_test.shape[0], 128*128)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(df_train.shape)
print(df_val.shape)

# reshape images to specify that it's a single channel
X_val = X_val.reshape(X_val.shape[0], 128, 128, 1)
X_train = X_train.reshape(X_train.shape[0], 128, 128, 1)
X_test = X_test.reshape(X_test.shape[0], 128, 128, 1)

(10000, 16384)
(1000, 16384)
(1500, 16384)
(10000, 9)
(1000, 9)


In [10]:
def buildModel(dropout1,dropout2,dropout3):
  model = keras.Sequential()
  # 32 convolution filters used each of size 3x3
  model.add(Conv2D(16, kernel_size=(6, 6), activation='relu', input_shape=(128, 128, 1)))
  model.add(Conv2D(32, kernel_size=(4, 4), activation='relu'))
  model.add(MaxPooling2D(pool_size=(4, 4)))
  model.add(Conv2D(64, kernel_size=(5, 5), activation='relu'))
  model.add(MaxPooling2D(pool_size=(5, 5)))
  model.add(Dropout(dropout1))
  model.add(Flatten())
  # fully connected to get all relevant data
  model.add(Dense(64, activation='relu'))
  # one more dropout
  model.add(Dropout(dropout2))
  # output a softmax to squash the matrix into output probabilities
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='Adam', 
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                metrics=['accuracy'])
  model.summary()

  return model

In [15]:
def test(element):
  Adf_train = pd.read_csv("HW4_data/public/train_label.csv", index_col=0)
  AX_train = X_train
  print(element)
  print(Adf_train[(Adf_train[element]==0)].iloc[:].shape)
  print(Adf_train[(Adf_train[element]==1)].iloc[:].shape)
  Adf_train = pd.concat([Adf_train[(Adf_train[element]==1)], Adf_train[(Adf_train[element]==0)].iloc[:Adf_train[(Adf_train[element]==1)].iloc[:].shape[0]+300]]).sample(frac=1)
  AX_train = AX_train[Adf_train.index]
  print(element, ' has ', int(Adf_train[(Adf_train[element]==1)].iloc[:].shape[0]), ' #0~~')
  return int(Adf_train[(Adf_train[element]==1)].iloc[:].shape[0]), Adf_train, AX_train

In [16]:
def validRes(element,model):
  prediction = model.predict(X_val)
  print(prediction.shape)
  for i in range(0,prediction.shape[0]):
    if prediction[i]>0.5: 
      prediction[i] = 1
    else:
      prediction[i] = 0
  print(element, ' f1: ',f1_score(df_val[element],prediction))
  return f1_score(df_val[element],prediction)

In [17]:
def genPredict(element,model):
  basicPredictRes = model.predict(X_test)
  for i in range(0,basicPredictRes.shape[0]):
    if basicPredictRes[i]>0.5: 
      basicPredictRes[i] = 1
    else:
      basicPredictRes[i] = 0
  return basicPredictRes

In [18]:
dropout1, dropout2, dropout3 = 0.2, 0.2, 0.2
normalEpoch = 45
sparseEpoch = 35
batchSize = 32
validationSplit = 0.3
sparseDef = 1400

In [19]:
targets = ['Atelectasis',	'Cardiomegaly',	'Edema',	'Lung Opacity',	'No Finding',	'Pleural Effusion',	'Support Devices']

STUDENT_ID = '108062138'
output_path = STUDENT_ID + '_prediction.csv'

res = {}
f1s = {}
for element in targets:
  oneSize, Adf_train, AX_train = test(element)
  model = buildModel(dropout1, dropout2, dropout3)
  if oneSize > sparseDef:
    history = model.fit(X_train, df_train[element], epochs=normalEpoch, batch_size=batchSize, validation_split=validationSplit)    
  else:
    history = model.fit(AX_train, Adf_train[element], epochs=25, batch_size=batchSize, validation_split=validationSplit)
  test_loss, test_acc = model.evaluate(X_val, df_val[element])
  print('Test accuracy:', test_acc)

  f1 = validRes(element,model)
  f1s[element] = f1
  basicPredictRes = genPredict(element,model)

  res[element] = basicPredictRes
  df = pd.DataFrame(basicPredictRes)
  df.columns = [element]
  specificOutputPath = element + output_path
  df.to_csv('./output/' + specificOutputPath, encoding='utf-8',header = True)

Atelectasis
(9264, 9)
(736, 9)
Atelectasis  has  736  #0~~
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 123, 123, 16)      592       
                                                                 
 conv2d_19 (Conv2D)          (None, 120, 120, 32)      8224      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 30, 30, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 26, 26, 64)        51264     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 5, 5, 64)         0         
 g2D)                                                            
                                                             

In [20]:
print(f1s)

{'Atelectasis': 0.2626865671641791, 'Cardiomegaly': 0.39294710327455923, 'Edema': 0.717687074829932, 'Lung Opacity': 0.21387283236994226, 'No Finding': 0.8854532677442024, 'Pleural Effusion': 0.4236111111111111, 'Support Devices': 0.46043165467625896}
